In [1]:
import setup as st
import fx_lib as fx
from datetime import datetime
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 40)
import numpy as np
import itertools
import pickle
import json
import yaml
with open(r'../inp/character_inputs.yaml') as file:
    target_stats = yaml.load(file, Loader=yaml.FullLoader)

with open('../inp/lellian_master.json') as json_file:
    data = json.load(json_file)

import set_combo as sc

In [2]:
st.GEAR_12 = 1             ##enhance all main stats to at least 12
st.NO_EQUIPPED_GEAR = 1    ## 0 = use equipped gear
st.GEAR_LIMIT = 16         ##maximum number of gear items for each slot
st.REPLACE_GEAR = 1        ##removes previously equipped gear and unlocks it after each character optimization
st.SELECTOR = 1            ##manually select which gear to use thorugh python interface
st.USE_CURR_EQUIP = 0      ##any currently equipped gear will be kept only

In [3]:
hero_order = target_stats['Hero_Order']
hero_order = [    
    'Faithless Lidica','Yufine','Angelic Montmorancy','Dizzy','Martial Artist Ken'
    ,'Seaside Bellona','Cermia','Top Model Luluca','Tenebria','Lilias','Krau','Tamarinne'
    ,'Alexa','Furious','Kayron','Challenger Dominiel','Charles','Ravi','Violet','Falconer Kluri'
    ,'Auxiliary Lots','Basar','Tywin','Destina','Vildred'
    ,'Sinful Angelica','General Purrgis','Vivian','Crimson Armin'
    ,'Adventurer Ras','Alencia','Roana','Leo','Commander Lorina','Baiken','Diene'
    
    ,'Benevolent Romann','Tempest Surin','Iseria','Sol','Yuna'
    ,'Fairytale Tenebria','Achates','Righteous Thief Roozid','Chloe','Lena'
    ,'Champion Zerato','Sigret','Desert Jewel Basar','Elena','Cidd','Melissa','Ken','Mort'
    ,'Luna','Landy','Celine','Cerise','Luluca','Blood Blade Karin','Kiris','Flan'
    ,'Free Spirit Tieria','Shooting Star Achates','Cecilia','Crescent Moon Rin','All-Rounder Wanda','Khawana'
    
    ,'Pavel','Kise','Kawerik','Judith','Sez','Armin','Lidica','Holiday Yufine','Mascot Hazel'
    ,'Blaze Dingo','Baal And Sezan','Elphelt','Surin','Silk','Karin','Serila','Mercenary Helga','Guider Aither'

    ,'Tieria','Schuri','Kizuna AI','Chaos Inquisitor','Assassin Cartuja','Maya','Taranor Guard','Researcher Carrot'

    ,'Ainos','Rose','Angelica','Celestial Mercedes','Assassin Cidd','Mistychain','Carmainerose','Jecht'
    ,'Rin','Clarissa','Adlay','Lots','Mirsa','Eaton','Coli'

    ,'Sonia','Glenn','Godmother','Ian'
    ,'Pyllis','Celeste','Hurado','Corvus','Crozet','Gloomyrain','Nemunas','Khawazu','Purrgis'
    ,'Roaming Warrior Leo','Sven','Otillie','Romann','Dominiel','Dingo','Requiemroar'
    ,'Bask','Aither','Zerato','Cartuja','Jena','Taranor Royal Guard'
    ,'Rikoris','Arowell','Elson','Batisse','Church of Ilryos Axe','Gunther','Kikirat V2'
    ]

In [4]:
df_hero = fx.hero_json_to_df(hero_order, data)
sc.df_items['start_loc'] = sc.df_items['hero']
if len(df_hero['Character'][df_hero['Atk'].isnull()]) > 0:
    print("Character data missing ", df_hero['Name'][df_hero['Atk'].isnull()].values)
    exit()

In [ ]:
# # lock specified list of heroes
lock_gear = target_stats['Lock_Gear']
lock_gear = [  
    ]
sc.df_items['reco'] = np.where( sc.df_items['hero'].isin(lock_gear) , sc.df_items['hero'], sc.df_items['reco'] )

In [101]:
# lock all equipped
# lock_gear = sc.df_items['hero'].unique()
# sc.df_items['reco'] = np.where( sc.df_items['hero'].isin(lock_gear) , sc.df_items['hero'], sc.df_items['reco'] )
# steal_gear
# steal_gear = []
# sc.df_items['reco'] = np.where( sc.df_items['hero'].isin(steal_gear) , '', sc.df_items['reco'] )


# sc.df_items['hero'] = np.where( sc.df_items['id'].isin(['jtxd83p6']) , '', sc.df_items['hero'] )

In [ ]:
print("")
if st.NO_EQUIPPED_GEAR == 1:
    print("No equipped gear will be included in optimization.  Pieces of gear: ", len(sc.df_items[(sc.df_items.hero == np.nan) | (sc.df_items.hero == '')]) )
elif len(target_stats['Lock_Gear']) > 0:
    print("Gear equipped on the following heros will not be included in optimization: ", target_stats['Lock_Gear'])
else:
    print("All gear will be used for optimization", len(sc.df_items) )
print("")
print("The following heroes will run: ", hero_order)

In [ ]:
sc.df_items[(sc.df_items.reco > '')].sort_values(by=['reco','Type'])

In [ ]:
xx = sc.df_items.groupby('hero').count().reset_index()
xx[(xx.hero > '')&(xx.efficiency < 6)][['hero','id']]

In [ ]:
main_list = [item for item in hero_order if item not in xx[(xx.hero > '')&(xx.efficiency == 6)].hero.values]
print(main_list)

### SELECT THE HERO

In [184]:
j=62
char = hero_order[j]
# char = 'Mort'
st.USE_CURR_EQUIP = 1      ##any currently equipped gear will be kept only

In [185]:
print("start hero: ", char, " // Level ", df_hero[df_hero.Name == char]['Lvl'].values , "  //  ", datetime.now())
if char in target_stats: char2 = char
elif char in target_stats["Type"]: char2 = target_stats['Type'][char]
else: char2 = 'General'
print("Build type: ", char2, "priority: ", target_stats[char2]['Prio'])

##what sets to use
input_sets = target_stats[char2]['input_sets']
exclude_sets = target_stats[char2]['exclude_sets']
include_sets = fx.gen_input_sets(input_sets, exclude_sets)
print(char, include_sets)

start hero:  Free Spirit Tieria  // Level  [60.]   //   2021-01-04 17:34:33.844354
Build type:  General priority:  ['SPD', 'EHP', 'Dmg_Rating']
Free Spirit Tieria ['Speed' 'Hit' 'Critical' 'Defense' 'Health' 'Attack' 'Resist' 'Revenge'
 'Injury' 'Penetration']


In [186]:
sc.df_items[(sc.df_items.start_loc == char)|(sc.df_items.reco == char)].sort_values(by=['reco','Type'])

,efficiency,enhance,hero,id,level,locked,mainStat,p_id,rarity,set,slot,subStat1,subStat2,subStat3,subStat4,grade,Type,ATK%,ATK,SPD,...,HP%,HP,DEF%,DEF,EFF,RES,main_tp,main_val,GR,GR2,minp,maxp,spdp,rating,efficiency2,max_eff,current_eff,current_pct,reco,start_loc
558,75,15,Free Spirit Tieria,jtczyxzh,67,False,"[Spd, 35]",id014526,Epic,Speed,Boots,"[CChance, 9]","[Res, 7]","[DefP, 17]","[CDmg, 10]",0,5,0,0,0,...,0,0,17,0,0,7,Spd,35,0.601849,0.601849,0.601849,0.601849,0,1.879849,18,75,7.360205,7.360205,,Free Spirit Tieria


In [173]:
# sorted(sc.df_items['reco'][~(sc.df_items.reco.isnull())].unique())
# sc.df_items[['reco','hero','Type']][(sc.df_items.hero == char)]

##### Change inputs (if required)

In [187]:
include_sets = [ 'Destruction','Critical']
# include_sets = [ 'Speed','Health']
# include_sets = ['Immunity','Speed','Critical','Resist','Health','Defense','Counter']
include_sets = [ 'Speed','Health','Defense','Resist','Hit','Critical']
# include_sets = ['Hit','Lifesteal','Resist','Defense','Health','Critical','Immunity','Penetration']
# include_sets = [ 'Critical','Rage','Attack','Destruction','Speed','Immunity','Counter','Resist','Health']
# include_sets = [ 'Critical','Health','Unity','Hit']
# include_sets = np.delete(include_sets,[5,6])
include_sets

['Speed', 'Health', 'Defense', 'Resist', 'Hit', 'Critical']

### Run Gear Combinations

In [188]:
st.GEAR_LIMIT = 22
only4_flag = 1
gear_comb_dict = fx.set_combo(sc.df_items[sc.df_items.set.isin(include_sets)], char, sc.l4, sc.l2)  ## Output gear_comb_dict[ [set_nm] , [type] , [ID] ]
# sc_output = sc.set_combination_iterate(gear_comb_dict, fx.set_4[fx.set_4.Set_Nm.isin(include_sets)].Set_Nm.values , fx.set_2[fx.set_2.Set_Nm.isin(include_sets)].Set_Nm.values)
sc_output = sc.set_combination_iterate(gear_comb_dict, fx.set_4[fx.set_4.Set_Nm.isin(include_sets)].Set_Nm.values , fx.set_2[fx.set_2.Set_Nm.isin(include_sets)].Set_Nm.values, only4_flag)
print('sc_out length ', len(sc_output) , "   ", datetime.now())

sc_out length  17802     2021-01-04 17:34:55.589863


In [189]:
sc_df = pd.DataFrame(sc_output, columns =['Set_1', 'Set_2', 'Set_3','Complete','Gear'])
sc_df['gear_list'] = sc_df.apply(lambda row: fx.gear_split(row) , axis=1)
sc_df[['0','1','2','3','4','5']] = pd.DataFrame(sc_df.gear_list.values.tolist(), index= sc_df.index)
sc_df = sc_df.drop_duplicates(['0','1','2','3','4','5'])

##### Add Current Gear List

In [190]:
## get current gear setting stats
current_gear = pd.DataFrame(columns = ['Set_1', 'Set_2', 'Set_3', 'Complete', 'Gear', 'gear_list', '0', '1', '2', '3', '4', '5' ], index = ['0'])
try:
    nix = 0
    for gid in range(0,6):
        current_gear[str(gid)] = sc.df_items[(sc.df_items.start_loc == char) & (sc.df_items.Type == gid)][['id']].values
        val = sc.df_items[(sc.df_items.start_loc == char) & (sc.df_items.Type == gid)][['reco']].values
        if (val > '') & (val!=char): nix = 1
    current_gear = fx.get_set_bonus(current_gear, sc.df_items)
    current_gear['Complete'] = 'X'
    sc_df = sc_df.append(current_gear)
    sc_df = sc_df.reset_index()
    no_equipped_flag = 0
except:
    no_equipped_flag = 1

##### Get stats for gear combo

In [191]:
### final stats for each combination
output = fx.get_combo_stats(sc_df, df_hero, fx.mainst_sum(sc_df, sc.df_items), fx.subst_sum(sc_df, sc.df_items), \
                            fx.set_sum(sc_df), fx.bonus_eqp_sum(df_hero[df_hero.Name == char]), char, target_stats[char2])
output['CATK'] = output.ATK * output.CDMG / 100
print("number of unique combinations ", len(sc_df), ",  ", len(output) , "   ", datetime.now() )

started function get combo stats:    2021-01-04 17:34:58.365848
number of unique combinations  17802 ,   17802     2021-01-04 17:34:58.486486


In [192]:
if no_equipped_flag == 0:
    choice_df = output.iloc[-1:,:].copy()
    output = output.sort_values(by = ['WW'], ascending = False )  ## faster than inplace
else:    
    output = output.sort_values(by = ['WW'], ascending = False )  ## faster than inplace
    choice_df = output.iloc[:2,:].copy()
odf = output.copy()

output2 = output.copy()
for stat in fx.set_df[fx.set_df.Bonus_Stat != 'NA'].Bonus_Stat.values:
    output2.drop( output2[(output2[stat]).astype(int) < target_stats[char2][stat]['Min']].index, inplace=True)
    output2.drop( output2[(output2[stat]).astype(int) > target_stats[char2][stat]['Max']].index, inplace=True)
print("combinations within min/max: ",len(output2))
if len(output2)==0:
    output2 = output.copy()

combinations within min/max:  17802


In [193]:
## get top options for key stats
if st.SELECTOR == 1:
    choice_df = choice_df.append(output.iloc[:3,:])
    for STAT in np.unique(target_stats[char2]['Prio']):
        choice_df = choice_df.append(output.sort_values(by = STAT, ascending = False ).iloc[:3,:] )
    choice_df = choice_df.append(output2.sort_values(by = ['WW'], ascending = False ).iloc[:3,:] )
    for STAT in np.unique(target_stats[char2]['Prio']):
        choice_df = choice_df.append(output2.sort_values(by = STAT, ascending = False ).iloc[:3,:] )
for i in range(0,len(target_stats[char2]['Prio'])):
    target = target_stats[char2]['Prio'][i]
    if target != 'EFF':  cut_off = output2[target].quantile(0.9)
    elif target == 'EFF':  cut_off = np.minimum(output2[target].quantile(0.9), 55)
    output2 = output2[ output2[target] >= cut_off ]
print("combinations after prio sortings ", len(output2) )

if st.SELECTOR == 1:
    choice_df = choice_df.append(output2.sort_values(by = ['WW'], ascending = False ).iloc[:3,:] )
    choice_df = choice_df.append(output2[(output2.Set_1.isin(['Unity','Immunity'])) | (output2.Set_2.isin(['Unity','Immunity'])) | \
            (output2.Set_3.isin(['Unity','Immunity']))].sort_values(by = ['WW'], ascending = False ).iloc[:3,:] )
    choice_df = choice_df.append(output2[(output2.Set_1.isin(['Counter','Lifesteal','Rage']))].sort_values(by = ['WW'], ascending = False ).iloc[:3,:] )
    for STAT in np.unique(target_stats[char2]['Prio']):
        choice_df = choice_df.append(output2.sort_values(by = STAT, ascending = False ).iloc[:3,:] )
    choice_df = choice_df.drop(axis = 1, columns = ['gear_list','Gear']).drop_duplicates()

combinations after prio sortings  31


#### REVIEW GEAR OPTIONS

In [194]:
choice_df[['Set_1','Set_2','Set_3','WW','ATK','HP','DEF','SPD','CRIT','CDMG','EFF','RES','Dmg_Rating','EHP','Complete']]

,Set_1,Set_2,Set_3,WW,ATK,HP,DEF,SPD,CRIT,CDMG,EFF,RES,Dmg_Rating,EHP,Complete
9687,Speed,Critical,None,88.96,1535,9179,1143,175,66,209,95,61,12,441,1
9736,Speed,Critical,None,88.47,1650,9887,1350,177,61,209,52,46,12,543,1
9690,Speed,Critical,None,88.41,1631,9432,1196,175,62,209,72,61,12,470,1
4041,Speed,Hit,None,77.36,2391,7141,913,199,62,184,54,21,19,288,1
4037,Speed,Hit,None,77.51,2031,7141,948,197,60,217,63,11,18,297,1
13700,Speed,Critical,None,78.30,1952,7739,990,183,70,230,34,7,18,332,1
15115,Speed,Critical,None,80.35,1735,12276,1275,185,55,164,25,16,11,644,1
9592,Speed,Critical,None,85.19,1717,11601,1362,181,52,177,29,46,11,642,1
16394,Speed,Health,None,78.40,1650,12959,1186,185,39,171,29,21,10,641,1
14600,Speed,Critical,None,73.54,1685,10197,914,210,60,173,34,35,13,412,1


##### Play with different filters to optimize the gear

In [195]:
print(char, 'ATK',target_stats[char2]['ATK'],'\n','CRIT',target_stats[char2]['CRIT'],'\n','CDMG',target_stats[char2]['CDMG'],'\n',
'SPD',target_stats[char2]['SPD'],'\n','HP',target_stats[char2]['HP'],'\n','DEF',target_stats[char2]['DEF'],'\n',
'EFF',target_stats[char2]['EFF'],'\n','RES',target_stats[char2]['RES'])

Free Spirit Tieria ATK {'Min': 0, 'Max': 999999, 'Weight': 1} 
 CRIT {'Min': 0, 'Max': 101, 'Weight': 1} 
 CDMG {'Min': 150, 'Max': 999, 'Weight': 1} 
 SPD {'Min': 100, 'Max': 999, 'Weight': 1} 
 HP {'Min': 0, 'Max': 99999, 'Weight': 1} 
 DEF {'Min': 0, 'Max': 9999, 'Weight': 1} 
 EFF {'Min': 0, 'Max': 101, 'Weight': 1} 
 RES {'Min': 0, 'Max': 101, 'Weight': 1}


In [164]:
op_cols = ['Complete','Set_1','Set_2','Set_3','WW','ATK','HP','DEF','SPD','CRIT','CDMG','EFF','RES',\
           'Dmg_Rating','EHP','CATK','PI','WR','CP']
odf[op_cols][(odf.SPD >= 130) & (odf.HP >= 1000) & (odf.DEF >= 100) & (odf.EFF >= 0) & (odf.RES >= 0)
             & (odf.CRIT >= 0) & (odf.CDMG >=  150) & (odf.ATK >= 100) & (odf.EHP >= 100)  & (odf.Dmg_Rating >= 0)
#               & (odf.Set_1.isin(['Rage','Counter','Lifesteal']))
#               & (odf.Set_1.isin(['Speed']))
#               & (odf.Set_2.isin(['Immunity','Unity']))
#              & (odf.Complete == 'X')
            ] \
    .sort_values(['Dmg_Rating'],ascending=False).head(20)

,Complete,Set_1,Set_2,Set_3,WW,ATK,HP,DEF,SPD,CRIT,CDMG,EFF,RES,Dmg_Rating,EHP,CATK,PI,WR,CP
3566,1,Attack,Critical,None,96.69,2775,9028,592,131,95,173,52,16,16,268,4800.75,96,130,54357.0
39838,1,Counter,Critical,None,87.00,2387,8005,667,146,52,225,4,28,15,258,5370.75,86,106,42376.0
3608,1,Attack,Critical,None,94.18,2730,8255,637,131,88,167,57,16,15,257,4559.10,94,128,51463.0
4308,1,Attack,Critical,None,95.21,2703,8422,633,131,55,215,33,45,15,261,5811.45,95,122,47779.0
67207,1,Counter,Critical,None,83.58,2633,7281,589,147,56,181,13,42,15,215,4765.73,83,110,43847.0
3823,1,Attack,Critical,None,95.66,2620,8938,571,130,100,172,47,30,15,259,4506.40,95,133,53703.0
3914,1,Attack,Critical,None,95.68,2547,8775,708,132,63,209,17,49,15,294,5323.23,95,122,48030.0
67011,1,Counter,Critical,None,87.24,2633,7547,648,142,56,191,13,42,15,238,5029.03,87,112,44598.0
39676,1,Counter,Critical,None,92.38,2360,7961,726,135,59,235,11,36,15,272,5546.00,92,114,44638.0
39670,1,Counter,Critical,None,90.66,2387,8272,726,141,52,235,4,28,15,282,5609.45,90,108,42988.0


In [99]:
odf[op_cols][odf.Complete == 'X']

/Users/zwj721/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,Complete,Set_1,Set_2,Set_3,WW,ATK,HP,DEF,SPD,CRIT,CDMG,EFF,RES,Dmg_Rating,EHP,CATK,PI,WR,CP


In [196]:
row_idx = 4041
sc.df_items[sc.df_items.id.isin(odf.loc[row_idx].gear_list)].sort_values(by=['Type'])

,efficiency,enhance,hero,id,level,locked,mainStat,p_id,rarity,set,slot,subStat1,subStat2,subStat3,subStat4,grade,Type,ATK%,ATK,SPD,...,HP%,HP,DEF%,DEF,EFF,RES,main_tp,main_val,GR,GR2,minp,maxp,spdp,rating,efficiency2,max_eff,current_eff,current_pct,reco,start_loc
60,75,12,,jt5vcjww,85,False,"[Atk, 360]",id112206,Heroic,Hit,Weapon,"[CDmg, 4]","[Spd, 8]","[AtkP, 19]","[Res, 4]",1,0,19,0,8,...,0,0,0,0,0,4,Atk,360,0.517281,0.517281,0.565614,0.613948,12,1.957781,19,81,6.328615,6.328615,,
100,66,6,,jtersp4u,85,False,"[HP, 1188]",id120004,Heroic,Speed,Helmet,"[CChance, 4]","[CDmg, 11]","[AtkP, 14]","[X, 0]",1,1,14,0,0,...,0,0,0,0,0,0,HP,1188,0.399690,0.399690,0.544690,0.689690,8,1.905190,19,85,4.419073,4.419073,,
139,61,6,,man0529018,85,False,"[Def, 132]",man0529018,Rare,Speed,Armor,"[HPP, 6]","[CChance, 13]","[X, 0]","[X, 0]",2,2,0,0,0,...,6,0,0,0,0,0,Def,132,0.314815,0.314815,0.459815,0.604815,8,1.820315,18,80,3.858896,3.858896,,
201,72,12,,jtwafj86,85,False,"[Atk, 360]",id004609,Epic,Hit,Necklace,"[Spd, 8]","[CChance, 9]","[Res, 10]","[CDmg, 9]",0,3,0,0,8,...,0,0,0,0,0,10,Atk,360,0.592900,0.592900,0.641234,0.689567,12,1.775800,17,78,6.244790,6.244790,,
225,64,12,,jtax9g0q,85,False,"[Atk, 360]",id214509,Heroic,Speed,Ring,"[DefP, 4]","[Eff, 16]","[Spd, 8]","[AtkP, 4]",1,4,4,0,8,...,0,0,4,0,16,0,Atk,360,0.474071,0.474071,0.522404,0.570738,12,1.656971,16,70,5.460517,5.460517,,
558,75,15,Free Spirit Tieria,jtczyxzh,67,False,"[Spd, 35]",id014526,Epic,Speed,Boots,"[CChance, 9]","[Res, 7]","[DefP, 17]","[CDmg, 10]",0,5,0,0,0,...,0,0,17,0,0,7,Spd,35,0.601849,0.601849,0.601849,0.601849,0,1.879849,18,75,7.360205,7.360205,,Free Spirit Tieria


#### SELECT FINAL GEAR SET

#### Save Results to DF

In [197]:
#gear_selected = output2.iloc[0,:]
gear_selected = output.loc[row_idx]
reco_list = gear_selected['gear_list']

In [198]:
try: reco_df.append(gear_selected)
except: reco_df = pd.DataFrame(gear_selected)

In [199]:
sc.df_items['reco'][(sc.df_items.reco == char)] = ''
sc.df_items['reco'] = np.where( sc.df_items.id.isin(reco_list) , char, sc.df_items['reco'])
if st.REPLACE_GEAR == 1:
    sc.df_items['hero'] = np.where((sc.df_items.hero == char)&(sc.df_items.hero != sc.df_items.reco),'',sc.df_items.hero)

/Users/zwj721/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [200]:
sc.df_items = sc.df_items.sort_values(by = ['hero','reco','Type','efficiency','enhance'])
sc.df_items.to_pickle('../outp/upd_items.pkl')

### Go Back to Run Next Hero

##### ------

### Push results back to input

In [201]:
sc.df_items[~(sc.df_items.reco=='')][['start_loc','hero','efficiency','rating','reco','Type','slot','set','level','rarity','enhance','mainStat','subStat1','subStat2','subStat3','subStat4']].to_csv('../reco/gear_reco.csv')

sc.df_items['hero'] = np.where( sc.df_items.reco!='', sc.df_items['reco'],sc.df_items['hero'])
sc.df_items.to_pickle('../outp/equip_potential.pkl')

reco_df.to_csv('../reco/hero_stats.csv')
sc.df_items = sc.df_items.sort_values(by = ['hero','Type','efficiency','enhance'])
export2 = sc.df_items[['efficiency','hero','enhance','slot','level','set','rarity','mainStat','subStat1','subStat2','subStat3','subStat4','id','p_id','locked']].to_dict('records')
with open('../outp/upd_items.json', 'w') as fp: json.dump(export2, fp)

### Fin

recoup from error

In [ ]:
import pandas as pd
df_items = pd.read_pickle('../outp/upd_items.pkl')

In [ ]:
df_items[df_items.id.isin(odf.loc[row_idx].gear_list)].sort_values(by=['Type'])

In [ ]:
df_items[df_items.reco!=''][['start_loc','hero','efficiency','rating','reco','Type','slot','set','level','rarity','enhance','mainStat','subStat1','subStat2','subStat3','subStat4']].to_csv('../reco/gear_reco.csv')

df_items['hero'] = np.where( df_items.reco!='', df_items['reco'],df_items['hero'])
df_items.to_pickle('../outp/equip_potential.pkl')

df_items = df_items.sort_values(by = ['hero','Type','efficiency','enhance'])
export2 = df_items[['efficiency','hero','enhance','slot','level','set','rarity','mainStat','subStat1','subStat2','subStat3','subStat4','id','p_id','locked']].to_dict('records')
with open('../outp/upd_items.json', 'w') as fp: json.dump(export2, fp)

### Gear analysis

In [ ]:
df_lvl85 = sc.df_items[(sc.df_items.level == 85)].copy()

In [ ]:
## look at level 85 gear
df_lvl85.groupby(['level','enhance'])[['efficiency','maxp']].agg(['count','max','mean','min']) 

In [ ]:
# df_lvl85[(df_lvl85.slot == 'Boots')&(df_lvl85.main_tp == 'Spd')].sort_values(by=['efficiency'], ascending = False)
df_lvl85[(df_lvl85.enhance < 15)&(df_lvl85.SPD > 0)].sort_values(by=['maxp'], ascending = False)

#### Gear var

In [ ]:
# gear_chk = sc.df_items[(sc.df_items.main_tp == 'Spd')&(sc.df_items.enhance<15)].copy()
gear_chk = sc.df_items[(sc.df_items.level >= 75)&(sc.df_items.Type.isin([3,4]))].copy()

In [ ]:
gear_chk.groupby(['slot','main_tp','set'])[['efficiency','maxp']].agg(['count','max','mean','min']) 

#### Gear list

In [ ]:
# gear_chk = sc.df_items[(sc.df_items.main_tp == 'Spd')&(sc.df_items.enhance<15)].copy()
gear_chk = sc.df_items[(sc.df_items.level >= 75)&(sc.df_items.enhance<15)].copy()

In [ ]:
gear_chk.sort_values(['SPD'],ascending=False)

### Gear removal

##### choose gear to delete

In [ ]:
df_items = pd.read_pickle('../outp/equip_potential.pkl')

In [ ]:
x1 = df_items['efficiency'].quantile(q=0.4)
x2 = df_items['max_eff'].quantile(q=0.3)
x3 = df_items['rating'].quantile(q=0.4)
x4 = df_items['current_eff'].quantile(q=0.3)

In [ ]:
low_items = df_items[(df_items.efficiency <= x1) & (df_items.max_eff <= x2) & (df_items.rating <= x3) & (df_items.current_eff <= x4) & (df_items.SPD <= 8)].copy()


In [ ]:
print("number of items" , len(df_items))
print("number of unequipped items" , len(df_items[df_items.hero=='']))
print("number of items to delete" , len(low_items))
print("number of unequipped items to delete" , len(low_items[low_items.hero=='']))

In [ ]:
low_items

In [ ]:
low_items[low_items.hero==''][['id','p_id','efficiency','max_eff','set','slot','level','enhance']].to_csv('../outp/remove_list.csv')

##### remove gear from items

In [ ]:
remove = pd.read_csv('../outp/remove_list.csv')

In [ ]:
check1 = len(remove.id.values)
check2 = df_items[df_items.id.isin(remove.id.values)]['id'].count()

In [ ]:
if check1 == check2:
    print('pass', check1, check2)
    df_items = df_items.sort_values(by = ['hero','Type','efficiency','enhance'])
    export1 = df_items[~df_items.id.isin(remove.id.values)][['efficiency','hero','enhance','slot','level','set','rarity','mainStat','subStat1','subStat2','subStat3','subStat4','id','p_id','locked']].copy()
    export2 = export1.to_dict('records')
    with open('../outp/upd_items.json', 'w') as fp: json.dump(export2, fp)

In [ ]:
print( sc.df_items['id'].count() , export1['id'].count(), check1 )